# Processament de dades

L'objectiu del tractament és obtenir una visualització de dades on es vegi la suma dels 10 millors castells de la temporada d'una colla enfront els anys des de la seva fundació. Per fer-ho haurem de combinar les 3 fonts de dades (actuacions_completes, puntuacions i colles_actuals).

## Anys des de la fundació

El primer que farem serà tenir en compte només colles actives de Catalunya no siguin universitàries i no estiguin en formació. 

Un cop fet això, crearem una columna 'year' on s'hi guardarà l'any on es va fer aquell castell. Posteriorment s'unificarà el conjunt d'actuacions amb les dades d'anys de fundació de les colles i es calcularà el temps des de la fundació.

In [355]:
import pandas as pd

In [356]:
df = pd.read_csv("actuacions_completes.csv")
df.head()

,id,date,ronda,intent,position,diada_name,show_longitude,show_latitude,colla_is_university,colla_is_training,...,colla_name,colla_pantone,colla_active,castell_type_name,castell_type_name_short,castell_type_estructura,castell_type_alcada,castell_result_name,city_name,city_country_id
0,280263,1926-07-26,1,0,0,NaN,NaN,NaN,0.0,0.0,...,Colla Nova dels Xiquets de Valls ( -1936),NaN,0.0,TRES DE SET,3de7,3.0,7.0,Intent,El Vendrell,1.0
1,280264,1926-07-26,2,0,1,NaN,NaN,NaN,0.0,0.0,...,Colla Nova dels Xiquets de Valls ( -1936),NaN,0.0,QUATRE DE SET,4de7,4.0,7.0,Intent,El Vendrell,1.0
2,280265,1926-07-26,3,0,2,NaN,NaN,NaN,0.0,0.0,...,Colla Nova dels Xiquets de Valls ( -1936),NaN,0.0,DOS / TORRE DE SIS,2de6,2.0,6.0,Intent,El Vendrell,1.0
3,280266,1926-07-26,4,0,3,NaN,NaN,NaN,0.0,0.0,...,Colla Nova dels Xiquets de Valls ( -1936),NaN,0.0,PILAR DE CINC,Pde5,1.0,5.0,Descarregat,El Vendrell,1.0
4,280287,1926-08-21,1,0,0,NaN,NaN,NaN,0.0,0.0,...,Colla Vella dels Xiquets de Valls,Vermelló-rosat,1.0,CASTELLS NO DOCUMENTATS,ND,NaN,NaN,Descarregat,L'Arboç,1.0


In [357]:
df = df[
    (df['colla_is_university'] == 0) &
    (df['colla_is_training'] == 0) &
    (df['colla_is_discontinuous'] == 0) &
    (df['colla_is_international'] == 0) &
    (df['colla_active'] == 1)
].drop(columns=['colla_is_university', 'colla_is_training', 'colla_is_discontinuous', 'colla_is_international', 'colla_active'])

In [358]:
foundation_df = pd.read_csv("colles_actuals.csv", )
df = df.merge(foundation_df, left_on="colla_name", right_on="Nom", how="left")
df = df.drop(columns=["Nom"])

df = df.rename(columns={"Fundació": "foundation_year"})
df["foundation_year"] = df["foundation_year"].astype(int)

In [359]:
df['year'] = df['date'].str[:4].astype(int)
df = df[df['year'] < 2025]
df['years_since_foundation'] = df['year'] - df['foundation_year']

df = df[df["years_since_foundation"]>=0] # Algunes colles s'han refundat. Agafem només els anys des de l'última refundació

## Puntuacions

Ara l'objectiu és trobar per cada colla la suma de punts dels seus 10 millors castells cada temporada. Per fer-ho, ens quedem només amb castells carregats o descarregats, revisem que les nomenclatures concordin entre conjunts de dades i un cop sigui així, els unim.

In [360]:
df = df[df['castell_result_name'].isin(["Carregat", "Descarregat"])]
df['castell_type_name_short'].unique()

array(['3de6', '4de6', 'Pde4', '2de5', 'ND', '2de6', 'Pde5s', 'Pde4s',
       '4de5', 'Pde5', '4de7', '3de7', '3de6s', '3de5n', '4de7p', '5de7',
       '3de6sn', '2de7', '3de7s', '5de6', '5de6n', '4de8', '4de6n',
       '2de6n', '3de6n', '4de5n', '4de6p', '4de5an', '3de8', '2de5n',
       'Pde4cam', 'Pde6', '4de7n', '5de8', 'Pde7f', '2de7f', 'Pde6f',
       '2de8f', '3de7n', '4de6pn', '3de5', 'Pde5f', '5de5n', '9de5n',
       '9de6n', '4de5a', 'Pde5n', 'Pde5cam', '4de9f', 'Pde4n', 'Pde6s',
       '3de7sn', '4de8p', '3de9f', '3de6p', 'Pde4sn', '5de5', '3de5s',
       '9de6', '3de7p', 'Pde4scam', '9de7', '6de6', 'Pde5scam', '2de9fm',
       '3de6pn', 'Pde8fm', '5de9f', '4de9fp', '5de7n', '3de8f', '6de7',
       '4de9', '3de10fm', '2de8', '3de8s', '2de6s', '7de6', '9de8',
       '2de9f', '3de8p', '5de7p', '12de7', '3de9fp', '2de7s', '7de7', nan,
       '5de8p', '5de6p', '7de8', '4de7s', '4de6s', '7de6pp', '7de6n',
       '12de6', '7de9f', '8de6', '4de7pn', '3de7pn', '6de7p', '10de7',
    

In [361]:
puntuacions = pd.read_csv("puntuacions.csv")
puntuacions['Castell'].unique()

array(['Pde4', '4de6', '3de6', '3de6p', '4de6p', '7de6', '5de6', '7de6p',
       '5de6p', '3de6s', '2de6', 'Pde5', '9de6', '4de7', '3de7', '3de7p',
       '4de7p', '7de7', '5de7', '7de7p', '5de7p', '3de7s', '9de7', '2de7',
       '4de8', 'Pde6', '3de8', '7de8', '2de8f', 'Pde7f', '5de8', '4de8p',
       '3de8p', '7de8p', '5de8p', '4de9f', '3de9f', '9de8', '3de8s',
       '2de9fm', 'Pde8fm', '7de9f', '5de9f', '4de9fp', '3de9fp', '4de9',
       '2de8', '3de10fm', '4de10fm', '9de9f', '2de9f', 'Pde9fmp', '3de9',
       '2de10fmp', '4de10f', '3de10f'], dtype=object)

In [362]:
# Veiem que els noms dels castells quadren menys per castells com pilars caminats o al balcó, però són castells que no puntuen
# El següent que hem de fer és pivotar el dataset de puntuacions, passant de tenir 3 columnes: Castell, Carregat, Descarregat i obtenint així 3 columnes: castell, resultat, puntuació
puntuacions = puntuacions.melt(id_vars=["Castell"], value_vars=["Carregat", "Descarregat"], var_name="resultat", value_name="puntuacio")

In [363]:
df = df.merge(puntuacions, left_on=["castell_type_name_short", "castell_result_name"], right_on=["Castell", "resultat"], how="left")

In [364]:
df[df['puntuacio'].isna()]['castell_type_name_short'].unique()

array(['2de5', 'ND', 'Pde5s', 'Pde4s', '4de5', '3de5n', '3de6sn', '5de6n',
       '4de6n', '2de6n', '3de6n', '4de5n', '4de5an', '2de5n', 'Pde4cam',
       '4de7n', '2de7f', 'Pde6f', '3de7n', '4de6pn', '3de5', 'Pde5f',
       '5de5n', '9de5n', '9de6n', '4de5a', 'Pde5n', 'Pde5cam', 'Pde4n',
       'Pde6s', '3de7sn', 'Pde4sn', '5de5', '3de5s', 'Pde4scam', '6de6',
       'Pde5scam', '3de6pn', '5de7n', '3de8f', '6de7', '2de6s', '12de7',
       '2de7s', nan, '4de7s', '4de6s', '7de6pp', '7de6n', '12de6', '8de6',
       '4de7pn', '3de7pn', '6de7p', '10de7', '8de7', 'pd4balcó', '10de8',
       'pd5balcó', 'pd4cambalcó', 'pd5cambalcó'], dtype=object)

Veiem que els castells que han quedat sense puntuacions són castells que no estan puntuats per part de la coordinadora. Vist això, eliminem els registres sense puntuació i busquem els 10 millors castells per temporada i colla.

In [365]:
df = df.dropna(subset=['puntuacio'])
df

,id,date,ronda,intent,position,diada_name,show_longitude,show_latitude,colla_id_string,colla_name,...,castell_type_alcada,castell_result_name,city_name,city_country_id,foundation_year,year,years_since_foundation,Castell,resultat,puntuacio
0,280295,1926-10-15,1,0,0,NaN,NaN,NaN,nensdelvendrell,Nens del Vendrell,...,6.0,Descarregat,El Vendrell,1.0,1926,1926,0,3de6,Descarregat,96.0
1,280296,1926-10-15,2,0,1,NaN,NaN,NaN,nensdelvendrell,Nens del Vendrell,...,6.0,Descarregat,El Vendrell,1.0,1926,1926,0,4de6,Descarregat,91.0
2,280297,1926-10-15,4,0,2,NaN,NaN,NaN,nensdelvendrell,Nens del Vendrell,...,4.0,Descarregat,El Vendrell,1.0,1926,1926,0,Pde4,Descarregat,69.0
5,280320,1927-02-03,1,0,0,NaN,NaN,NaN,nensdelvendrell,Nens del Vendrell,...,6.0,Descarregat,Molins de Rei,1.0,1926,1927,1,2de6,Descarregat,200.0
6,280321,1927-07-10,1,0,0,NaN,NaN,NaN,nensdelvendrell,Nens del Vendrell,...,4.0,Descarregat,El Vendrell,1.0,1926,1927,1,Pde4,Descarregat,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249743,316334,2024-12-08,1,1,1,NaN,NaN,NaN,encantats,Colla Castellera Els Encantats de Begues,...,4.0,Descarregat,Castelldefels,1.0,2012,2024,12,Pde4,Descarregat,69.0
249744,316335,2024-12-08,2,1,2,NaN,NaN,NaN,encantats,Colla Castellera Els Encantats de Begues,...,4.0,Descarregat,Castelldefels,1.0,2012,2024,12,Pde4,Descarregat,69.0
249745,316578,2024-12-15,1,1,1,NaN,NaN,0.0,castellersdecaldes,Castellers de Caldes de Montbui,...,4.0,Descarregat,Caldes de Montbui,1.0,1996,2024,28,Pde4,Descarregat,69.0
249746,316472,2024-12-17,1,1,1,NaN,NaN,NaN,collajoves,Colla Joves Xiquets de Valls,...,5.0,Descarregat,Valls,1.0,1970,2024,54,Pde5,Descarregat,210.0


In [366]:
# Get best 10 castells per colla and year
df['puntuacio'] = df['puntuacio'].astype(int)
df = df.sort_values(by=['colla_name', 'year', 'puntuacio'], ascending=[True, True, False])
df = df.groupby(['colla_name', 'year']).head(10).reset_index(drop=True)
df

,id,date,ronda,intent,position,diada_name,show_longitude,show_latitude,colla_id_string,colla_name,...,castell_type_alcada,castell_result_name,city_name,city_country_id,foundation_year,year,years_since_foundation,Castell,resultat,puntuacio
0,196544,1997-04-20,4,0,3,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Manacor,1.0,1997,1997,0,3de6,Descarregat,96
1,196549,1997-04-20,4,0,3,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Santa Eugenia,1.0,1997,1997,0,3de6,Descarregat,96
2,201448,1997-10-04,1,0,0,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Artà,1.0,1997,1997,0,3de6,Descarregat,96
3,201584,1997-10-05,2,0,5,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Artà,1.0,1997,1997,0,3de6,Descarregat,96
4,202937,1997-11-23,1,0,0,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Manacor,1.0,1997,1997,0,3de6,Descarregat,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20367,313330,2024-09-21,3,1,3,NaN,0.581194,40.713024,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,Amposta,1.0,2013,2024,11,3de6p,Descarregat,119
20368,306459,2024-04-20,2,1,2,NaN,NaN,NaN,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,Gelida,1.0,2013,2024,11,3de6,Descarregat,96
20369,307277,2024-05-04,2,1,1,NaN,0.521925,40.820328,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,Tortosa,1.0,2013,2024,11,3de6,Descarregat,96
20370,309129,2024-06-09,3,1,3,NaN,NaN,NaN,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,El Prat de Llobregat,1.0,2013,2024,11,3de6,Descarregat,96


## Emmagatzematge

Guardem les dades en format .csv però només de les columnes que ens poden ser interessants per la visualització

In [367]:
df.columns

Index(['id', 'date', 'ronda', 'intent', 'position', 'diada_name',
       'show_longitude', 'show_latitude', 'colla_id_string', 'colla_name',
       'colla_pantone', 'castell_type_name', 'castell_type_name_short',
       'castell_type_estructura', 'castell_type_alcada', 'castell_result_name',
       'city_name', 'city_country_id', 'foundation_year', 'year',
       'years_since_foundation', 'Castell', 'resultat', 'puntuacio'],
      dtype='object')

In [368]:
# Busquem les dades només per colles que han fet almenys un castell de 8. Després d'això, agruparem per colla i any i sumarem les puntuacions
# colles_with_9plus = df[df['castell_type_alcada'] >= 9]['colla_name'].unique()
# df = df[df['colla_name'].isin(colles_with_9plus)]

grouped_df = df.groupby(['colla_name', 'year']).agg({
    'puntuacio': 'sum',
    'colla_pantone': 'first',
    'years_since_foundation': 'first',
}).reset_index()

#Before saving, we need to pivot the dataframe in order to have years_since_foundation as the first column and then each colla as a column
grouped_df = grouped_df.pivot(index='years_since_foundation', columns='colla_name', values='puntuacio').reset_index()
grouped_df.to_csv("castells_totes.csv", index=False)

In [369]:
df

,id,date,ronda,intent,position,diada_name,show_longitude,show_latitude,colla_id_string,colla_name,...,castell_type_alcada,castell_result_name,city_name,city_country_id,foundation_year,year,years_since_foundation,Castell,resultat,puntuacio
0,196544,1997-04-20,4,0,3,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Manacor,1.0,1997,1997,0,3de6,Descarregat,96
1,196549,1997-04-20,4,0,3,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Santa Eugenia,1.0,1997,1997,0,3de6,Descarregat,96
2,201448,1997-10-04,1,0,0,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Artà,1.0,1997,1997,0,3de6,Descarregat,96
3,201584,1997-10-05,2,0,5,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Artà,1.0,1997,1997,0,3de6,Descarregat,96
4,202937,1997-11-23,1,0,0,NaN,NaN,NaN,alotsdellevant,Al·lots de Llevant,...,6.0,Descarregat,Manacor,1.0,1997,1997,0,3de6,Descarregat,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20367,313330,2024-09-21,3,1,3,NaN,0.581194,40.713024,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,Amposta,1.0,2013,2024,11,3de6p,Descarregat,119
20368,306459,2024-04-20,2,1,2,NaN,NaN,NaN,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,Gelida,1.0,2013,2024,11,3de6,Descarregat,96
20369,307277,2024-05-04,2,1,1,NaN,0.521925,40.820328,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,Tortosa,1.0,2013,2024,11,3de6,Descarregat,96
20370,309129,2024-06-09,3,1,3,NaN,NaN,NaN,xiquelos,Xiqüelos i Xiqüeles del Delta,...,6.0,Descarregat,El Prat de Llobregat,1.0,2013,2024,11,3de6,Descarregat,96


In [370]:
# Treiem també un csv amb colla - color de camisa
colla_colors = df[['colla_name', 'colla_pantone']].drop_duplicates()
colla_colors.to_csv("colla_colors.csv", index=False)

In [371]:
# Pivot the dataframe to have 'year' as the index and each colla as a column
pivot_by_year = df.groupby(['year', 'colla_name'])['puntuacio'].sum().unstack('colla_name')
pivot_by_year = pivot_by_year.reset_index()
pivot_by_year.to_csv("castells_by_year_totes.csv", index=False)